In [1]:
import os
try:
    user_paths = os.environ['PYTHONPATH'].split(os.pathsep)
except KeyError:
    user_paths = []
    
print(user_paths)

['C:\\Code\\USD\\USD\\lib\\python']


In [1]:
from pxr import Usd, UsdGeom
import os

# Create simple scene with one sphere

In [2]:
# Execute only if you have already created a stage and need to redo it.
try:
    del stage
except NameError:
    pass

In [4]:
usd_scene_dir = '..\usd_scenes'
usd_sphere_filename='sphere.usda'
usd_sphere_filepath = os.path.join(usd_scene_dir, usd_sphere_filename)

In [14]:
try:
    stage = Usd.Stage.CreateNew(usd_sphere_filepath)
except:
    os.remove(usd_sphere_filepath)
    stage = Usd.Stage.CreateNew(usd_sphere_filepath)
    

# xformPrim = UsdGeom.Xform.Define(stage, '/world')
# spherePrim = UsdGeom.Sphere.Define(stage, '/world/sphere')

sphere = stage.DefinePrim('/sphere', 'Xform')
geom = stage.DefinePrim('/sphere/geom', 'Sphere')

stage.SetDefaultPrim(sphere)
stage.GetRootLayer().Save()

True

In [15]:
print stage.GetRootLayer().ExportToString()

#usda 1.0
(
    defaultPrim = "sphere"
)

def Xform "sphere"
{
    def Sphere "geom"
    {
    }
}




In [5]:
!usdview {usd_sphere_filepath}

State file not found, a new one will be created.


# Open and existing usd scene and inspect it

1. Access Xform and Sphere as Primitives
2. List attributes

In [6]:
stage = Usd.Stage.Open(usd_sphere_filepath)
sphere = stage.GetPrimAtPath('/sphere')
geom = stage.GetPrimAtPath('/sphere/geom')
sphere.GetPropertyNames()
geom.GetPropertyNames()

['doubleSided',
 'extent',
 'orientation',
 'primvars:displayColor',
 'primvars:displayOpacity',
 'proxyPrim',
 'purpose',
 'radius',
 'visibility',
 'xformOpOrder']

### Access primitive attributes

In [7]:
extentAttr = geom.GetAttribute('extent')
extentAttr.Get()
radiusAttr = geom.GetAttribute('radius')
radiusAttr.Set(2)
extentAttr.Set(extentAttr.Get()*2)

True

### Access Primitive attributes via schema

In [8]:
sphereSchema = UsdGeom.Sphere(geom)
color = sphereSchema.GetDisplayColorAttr()
color.Set([(0,0,1)])

True

### Apply a translation to the Xform

In [9]:
UsdGeom.XformCommonAPI(sphere).SetTranslate((5, 5, 5))
print stage.GetRootLayer().ExportToString()

#usda 1.0
(
    defaultPrim = "sphere"
)

def Xform "sphere"
{
    double3 xformOp:translate = (5, 5, 5)
    uniform token[] xformOpOrder = ["xformOp:translate"]

    def Sphere "geom"
    {
        float3[] extent = [(-2, -2, -2), (2, 2, 2)]
        color3f[] primvars:displayColor = [(0, 0, 1)]
        double radius = 2
    }
}




In [10]:
stage.GetRootLayer().Save()

True

In [11]:
!usdview {usd_sphere_filepath}

# Create a Reference Scene

In [23]:
usd_ref_scene_filename ='ref_scene.usda'
usd_ref_scene_filepath = os.path.join(usd_scene_dir, usd_ref_scene_filename)

In [28]:
if ref_stage:
    del ref_stage

In [29]:
ref_stage = Usd.Stage.CreateNew(usd_ref_scene_filepath)
ref_sphere = ref_stage.OverridePrim('/refSphere')
print ref_stage.GetRootLayer().ExportToString()

#usda 1.0

over "refSphere"
{
}




In [30]:
ref_sphere.GetReferences().AddReference(os.path.abspath(usd_sphere_filepath))
#ref_sphere.GetReferences().AddReference('./sphere.usda')
print ref_stage.GetRootLayer().ExportToString()
ref_stage.GetRootLayer().Save()

#usda 1.0

over "refSphere" (
    prepend references = @D:\PROJECTS\Code\USD\usd_testing\usd_scenes\sphere.usda@
)
{
}




True

In [31]:
!usdview {os.path.abspath(usd_ref_scene_filepath)}

## Reset transform to Identity
1. Trick: Set transform order array to empty

In [32]:
ref_Xform = UsdGeom.Xformable(ref_sphere)
ref_Xform.SetXformOpOrder([])
print ref_stage.GetRootLayer().ExportToString()
ref_stage.GetRootLayer().Save()

#usda 1.0

over "refSphere" (
    prepend references = @D:\PROJECTS\Code\USD\usd_testing\usd_scenes\sphere.usda@
)
{
    uniform token[] xformOpOrder = []
}




True

In [33]:
!usdview {os.path.abspath(usd_ref_scene_filepath)}

## Reference Another Sphere

In [34]:
ref_sphere2 = ref_stage.OverridePrim('/refSphere2')
ref_sphere2.GetReferences().AddReference(os.path.abspath(usd_sphere_filepath))
print ref_stage.GetRootLayer().ExportToString()
ref_stage.GetRootLayer().Save()

#usda 1.0

over "refSphere" (
    prepend references = @D:\PROJECTS\Code\USD\usd_testing\usd_scenes\sphere.usda@
)
{
    uniform token[] xformOpOrder = []
}

over "refSphere2" (
    prepend references = @D:\PROJECTS\Code\USD\usd_testing\usd_scenes\sphere.usda@
)
{
}




True

In [35]:
!usdview {usd_ref_scene_filepath}